**Install dependencies**

In [2]:
import yfinance as yf
import pg8000.native
import ssl
from pg8000 import Cursor

In [5]:
# get Tesla stock information
tsla = yf.Ticker("TSLA")

# display trading and company information
tsla.info

# get historical market data
df = tsla.history(period="max")

# use index as column (index represents dates)
df['Date'] = df.index

# number of days
len(df)

# use only the first 100 rows
df = df[:100]

# show dataframe
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
Date,,,,,,,,
2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0,2010-06-29
2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0,2010-06-30
2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0,2010-07-01
2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0,2010-07-02
2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0,2010-07-06
...,...,...,...,...,...,...,...,...
2010-11-11,5.720,5.820,5.466,5.608,9726500,0,0.0,2010-11-11
2010-11-12,5.650,6.100,5.614,5.968,13645500,0,0.0,2010-11-12
2010-11-15,6.044,6.588,6.044,6.160,13114500,0,0.0,2010-11-15


**Credentials**

In [ ]:
# provide credentials obtained from hoster
user = 'user-1234'
database = 'database-1234'
host = 'database.hoster.com'
password = 'password-1234'

**Connect to database**

In [125]:
# establish connection to database
conn = pg8000.native.Connection(user=user, database=database, host=host,
                              password=password)

In [127]:
# define query to create schema and table in database
sql_query ="""
CREATE SCHEMA test;
CREATE TABLE test.stocks(
        "date" date,
        "closing_price" float);
    """
conn.run(sql_query)

[]

In [149]:
# test whether schema was created successfully
conn.run("SELECT schema_name FROM information_schema.schemata")

[]

In [151]:
# define query to insert data into table
# create list of lists to transmit results to database

s = """INSERT INTO test.stocks_3(date, closing_price) VALUES (%s, %s)"""

obj = []
for index in range(len(df)):
    obj.append([df.Date[index],
                df.Close[index]])
            
    
cursor = Cursor(conn)
cursor.executemany(s, obj)
conn.run("COMMIT")

[]

In [154]:
# check data in table
for row in conn.run("SELECT * FROM test.stocks")[:5]:
    print(row)

[datetime.date(2001, 7, 19), 11.3179740905762]
[datetime.date(2001, 7, 20), 11.1985969543457]
[datetime.date(2001, 7, 23), 11.1911392211914]
[datetime.date(2001, 7, 24), 11.0866861343384]
[datetime.date(2001, 7, 25), 11.1538343429565]
